In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import data_process
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import cv2

In [4]:
data_process.data_to_tfrecord(imagepath='../data/images/', outputpath='../data/face_data7.tfrecord')

이미지를 tfrecord에 저장중: 100%|██████████| 1038/1038 [00:01<00:00, 519.29it/s]


이미지 갯수:  1038


In [9]:
filenames = '../data/face_data_7.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),    
    'class': tf.io.FixedLenFeature([], tf.int64),
    'landmark_points': tf.io.FixedLenFeature([], tf.float32),    
}

def _parse_image_function(example_proto): 
  # Parse the input tf.train.Example proto using the dictionary above.  
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset element_spec={'class': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(), dtype=tf.string, name=None), 'landmark_points': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [ ]:
for image_features in parsed_image_dataset:
    print(image_features)
    image_raw = image_features['class'].numpy()
print(image_raw)
# display.display(display.Image(data=image_raw))

In [5]:
@tf.function
def landmark_tfrecord_reader(example):
    feature_description = {"image": tf.io.VarLenFeature(dtype=tf.string),
                           "class": tf.io.VarLenFeature(dtype=tf.int64),
                           "landmark_points": tf.io.VarLenFeature(dtype=tf.float32)}

    example = tf.io.parse_single_example(example, feature_description)
    image_raw = tf.sparse.to_dense(example["image"])[0]
    image = tf.io.decode_png(image_raw, channels=3)
    image_class = tf.sparse.to_dense(example["class"])
    image_landmark = tf.sparse.to_dense(example["landmark_points"])

    return image, image_class, image_landmark